En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Llamada del archivo para su procesamiento

In [1]:
file_path = r"D:\\Personal\\Prueba_Tecnica\\LATAM\\farmers-protest-tweets-2021-2-4.json"

In [ ]:
#pip install py-spy
#pip install pyspy
#pip install memory-profiler
#pip install emoji

In [8]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import cProfile
from typing import List, Tuple
from datetime import datetime
from memory_profiler import memory_usage
import emoji
from collections import Counter

## Validaciones iniciales

In [ ]:
#Cara del df 

data = []
# Crear conjunto para la claves únicas 
unique_keys = set()

with open(file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line.strip())
        data.append(json_data)
        unique_keys.update(json_data.keys())

#Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
df_1 = pd.DataFrame(data).reindex(columns=unique_keys)

**Duplicados**

La validación de duplicados se realiza sobre la columna ID, dado que esta la que contiene los registros únicos de los twits

In [ ]:
df_1['id'].duplicated().sum()

**Vacíos**

In [ ]:
df_1.isnull().sum()/len(df_1)

Respecto a esto se identifca que la columna retweetedTweet tiene el 100% de datos nulos, en su momento si para algún punto se consideran eliminar las columna con alto % de vacío se procederá, de lo contrario permanecerá.

Por ejemplo, las columans mentioneUsers se podría esperar un alto % de vacíos dado que en todos los comentarios no necesariamente se etiqueta otra cuenta de "X".

## Punto 1 datos temporales

### **Función inicial**

**Consideraciones con las que se creó la función**

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas.

2. Se agrega una nueva columna al DataFrame para almacenar las fechas en formato date.

3. La columna de usuario se normaliza para extraer los datos pertinentes, ya que contiene información sobre los usuarios que realizaron las interacciones.

4. Para reducir el tamaño del DataFrame y optimizar el procesamiento, se crea un DataFrame más pequeño que incluye solo las columnas esenciales, como el ID de la interacción y la fecha, junto con los datos del usuario en columnas separadas.

5. Se utiliza groupby para crear una tabla que contenga las fechas, los usuarios y el recuento de interacciones por usuario en cada fecha.

6. Se crea una segunda tabla que calcula el total de interacciones por fecha, lo que permite ordenar la tabla por la fecha de manera efectiva.

7. Se uner la tabla del punto 5 y 6 y e ordena la tabla resultante de manera que las fechas con mayor actividad aparezcan primero, y dentro de cada fecha, los usuarios más activos se encuentren en la parte superior.

8. Se selecciona el primer registro de cada fecha, lo que proporciona el nombre de usuario con la mayor cantidad de interacciones para esa fecha en particular.

9. Finalmente, se crea una lista de tuplas que contiene las fechas y los nombres de usuario correspondientes, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.

In [ ]:
def q1_return_inicial(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    df = pd.read_json(file_path, lines=True)
    
    # Modificar la columna de fechas
    df['date_tw'] = pd.to_datetime(df['date']).dt.date
    
    # Normalizar la columna 'user' para obtener datos de usuarios
    df_user = json_normalize(df['user'])
    
    # Unir el DataFrame original con los datos normalizados de usuarios
    df = pd.concat([df[['id','date_tw']], df_user.add_prefix('user.')], axis=1)
    
    # Agrupar por fecha y usuario, contar las interacciones y obtener el total de interacciones por fecha
    df_tw_u = df.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')
    
    # Unir los DataFrames y ordenar por conteo total y por usuario
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw') \
                .sort_values(by=['count_total', 'count_user'], ascending=[False, False])
    
    # Seleccionar el usuario con más interacciones para cada fecha
    df_tw_u_f = df_tw_u.groupby('date_tw').first().reset_index().sort_values(by=['count_total'], ascending=False)
    
    # Seleccionar las primeras 10 filas y crear una lista de tuplas de fecha y usuario
    lista_resultado = list(zip(df_tw_u_f['date_tw'], df_tw_u_f['user.username']))[:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [ ]:
q1_return_inicial(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q1_return_inicial(file_path)')

**Memoria total utilizada**

El ejecutar la función memory_usage((q1_return_inicial, (file_path,))), se obtiene la memoria usada durante la ejecución de la función q1_return_inicial, así que para obtener el valor total de memoria utilizada se realiza la suma de los valores de la lista. 

In [ ]:
np.sum(memory_usage((q1_return_inicial, (file_path,))))

### **Intentos de optimización de la función inicial**

#### **Función 1**

Cambios:

1. En lugar de cargar todo el archivo JSON de una vez, se lee línea por línea para evitar cargar todo el archivo en memoria al mismo tiempo, lo que reduce el uso de memoria.

2. En lugar de usar add_prefix, se asignan los nombres de columnas directamente para mayor claridad.

3. El filtro para encontrar el usuario con mayor interaccion por fecha no se hace por método first(), si no que se usa encontrando los índices de estos usuarios y luego filtrando la tabla que contiene la información de fechas nombre de usarios, en esta misma tabla se filtran los 10 registros que pide el ejecicio.

4. Se crea un ciclo para la creación de la lista de datos requeridos. 

In [ ]:
def q1_return_v1(file_path: str) -> List[Tuple[datetime.date, str]] :
    # Punto 1: Lista para almacenar los datos JSON
    data = []

    # Punto 2: Conjunto para almacenar los nombres de las claves únicas
    unique_keys = set()

    # Punto 3: Abrir el archivo JSON y leer línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Punto 4: Cargar la línea como un objeto JSON
            json_data = json.loads(line.strip())
            
            # Punto 5: Agregar el objeto JSON a la lista
            data.append(json_data)
            
            # Punto 6: Actualizar el conjunto de claves únicas
            unique_keys.update(json_data.keys())

    # Punto 7: Crear DataFrame con los datos JSON
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    # Punto 8: Modificar la columna de fechas para obtener el formato AAAA-MM-DD
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date
    
    # Punto 9: Crear un dataframe con los datos expandidos de los usuarios que luego se une al df inicial
    df_user = json_normalize(df_inicial['user'])
    ## Se asigna el prefijo a los nombres de las columnas para no perder el origen
    df_user.columns = ['user.'+col for col in df_user.columns ]
    ## Se une el df de usuarios el df inicial y se eliminan los ínidices
    df_inicial = pd.concat([df_inicial[['id','date_tw']].copy(),df_user],axis=1).reset_index(drop=True)   
    
    # Punto 10: Se crean groupby para procesar solo la información de fechas y nombres de usuario, agregando la columna del conteo de interacciones de usuarios
    df_tw_u = df_inicial.groupby(['date_tw','user.username'])['id'].count().reset_index(name='count_user')
    
    # Punto 11: Se crea un df nuevo que contiene solo el total de interacciones por fecha.
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().sort_values(ascending=False).reset_index(name='count_total')
    ## Se unen el df original de fechas, usuarios, conteo de interacciones por usuario al df que contiene las fechas e interacciones por fecha, se ordena de tal forma que el conteo total y por usuario es descendente
    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total','count_user'],ascending=[False,False])
    ## Se buscan los índices de la primera fila de cada fecha.
    indeices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    ## Se filtra del df de twits de tal forma que se ordene de la fecha de mayor interaccion a la menor, solo los primeros 10 registros
    df_tw_u_f = df_tw_u.loc[indeices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10,::].reset_index(drop=True)
    
    # Punto 12: Se crea un ciclo para llenar una lista con la tuplas de fecha y el usuario con mayor cantidad de interacciones en X para ese día 
    lista_resultado = []
    for i in range(len(df_tw_u_f)):
        lista_resultado.append((df_tw_u_f['date_tw'][i],df_tw_u_f['user.username'][i]))
    
    return lista_resultado


**Resultado de ejecución del a función**

In [ ]:
q1_return_v1(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q1_return_v1(file_path)')

**Memoria total utilizada**

In [ ]:
np.sum(memory_usage((q1_return_v1, (file_path,))))

#### **Función 2**

Cambios:

Si bien se mantiene una estructura muy similar a la de la función 1, se realizan algunos pequeños cambios:

1. Al crear el concat entre el DataFrame inicila y el DataFrame de usuarios, en el segundo caso se usa solo la columna objetivo, que es el nombre de usuario.
2. La lista de resultados se realiza con una lista de compresión en iteración sobre las líneas del df final.

In [ ]:
def q1_return_t2(file_path: str) -> List[Tuple[datetime.date, str]]:
    data = []
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    df_inicial['date_tw'] = pd.to_datetime(df_inicial['date']).dt.date

    df_user = json_normalize(df_inicial['user'])
    df_user.columns = ['user.'+col for col in df_user.columns]
    df_inicial = pd.concat([df_inicial[['id','date_tw']], df_user['user.username']], axis=1).reset_index(drop=True)

    df_tw_u = df_inicial.groupby(['date_tw', 'user.username'])['id'].count().reset_index(name='count_user')
    df_tw_ug = df_tw_u.groupby('date_tw')['count_user'].sum().reset_index(name='count_total')

    df_tw_u = pd.merge(left=df_tw_u, right=df_tw_ug, how='left', on='date_tw').sort_values(by=['count_total', 'count_user'], ascending=[False, False])

    indices_max = df_tw_u.groupby('date_tw')['count_user'].idxmax()
    df_tw_u_f = df_tw_u.loc[indices_max].sort_values(by=['count_total'], ascending=False).iloc[0:10].reset_index(drop=True)

    lista_resultado = [(row['date_tw'], row['user.username']) for _, row in df_tw_u_f.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [ ]:
q1_return_t2(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q1_return_t2(file_path)')

**Memoria total utilizada**

In [ ]:
np.sum(memory_usage((q1_return_t2, (file_path,))))

## Punto 2 emojies

### **Función inicial**

**Consideraciones con las que se creó la función**

1. Primero, se carga el archivo JSON utilizando pd.read_json, teniendo en cuenta que está formado por múltiples líneas. La forma de carga adoptada tiene el sentido de no sobrecargar la memoria con una carga completa.

2. Las claves únicas tiene el sentido de asegurar que al conformar el dataframe luego de leer los datos van a estar encabezados de forma adecuada.

3. Se realiza el uso de la librería emoji y dentro de ella el atriburo EMOJI_DATA, el cual contiene la información asociada a los emojies.

4. Se crea una función para extraer los emojies de cada fila de contenido, de esta manera crea una lista de emojies para cada fila de content.

5. Se crea una columna dentro del dataFrame para poder almancenas los emojies encontrados en cada fila del content.

6. Se crea una lista de vacía de emojies a la cual se va integrando con las listas de los emojies encontrados en cada fila de la colimna 'emojis'

7. Se crea un dataframe con la lista de los emojies, luego se crea un nuevo df sobre este luego de hacer el conteo descente de aparición para cada emojie.

8. Se filtran los datos de color de los emojies, dado que por el momento, el enfoque se centra en identificar la forma básica de los emojis y no en considerar sus modificadores de color. Esto puede ayudar a evitar confusiones sobre el alcance y los objetivos del análisis actual.

9. Finalmente, se crea una lista de tuplas que contiene los emojies y la cantidad de apariciones, limitada a los primeros 10 registros para enfocarse en los usuarios más activos.

In [ ]:
def q2_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)
        
    df_emojies = pd.DataFrame(lista_emojies,columns=['emojie'])
    
    df_ef = pd.DataFrame(df_emojies.value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [ ]:
q2_run_inicial(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q2_run_inicial(file_path)')

**Memoria total utilizada**

In [ ]:
np.sum(memory_usage((q2_run_inicial, (file_path,))))

### **Intentos de optimización de la función inicial**

### **Función 1**

In [ ]:
def q2_run_v1(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    lista_emojies = []

    #Se crea un ciclo para obtener todos los emojies en una sola lista.
    for fila in df_inicial['emojis']:
        lista_emojies.extend(fila)

    df_ef = pd.DataFrame(pd.DataFrame(lista_emojies,columns=['emojie']).value_counts(),columns=['Cuenta']).reset_index()
    
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado


**Resultado de ejecución del a función**

In [ ]:
q2_run_v1(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q2_run_v1(file_path)')

**Memoria total utilizada**

In [ ]:
np.sum(memory_usage((q2_run_v1, (file_path,))))

In [ ]:
def q2_run_v2(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Usando la librería de emoji se crea una lista con los emojies "c" dentro un texto que se pasa, a la función
    def extraer_emojis(texto):
        return [c for c in texto if c in emoji.EMOJI_DATA]

    #Se aplica la función sobre la columna de contenido y se crea una nueva con el fin de procesar los emojies posteriormente
    df_inicial['emojis'] = df_inicial['content'].apply(extraer_emojis)
    
    # Crear una lista plana con todos los emojis encontrados en todas las filas
    lista_emojies = [emoji for fila in df_inicial['emojis'] for emoji in fila]
    
    # Crear DataFrame con los emojis y su frecuencia
    df_ef = pd.DataFrame(Counter(lista_emojies).most_common(), columns=['emojie', 'Cuenta'])
        
    #Se filtran los símbolos que representan la modificación de color de los emojies, se crea esta función bajo el supuesto que por ahora solo se necesita la figura del emojie no su color.
    df_ef = df_ef[~df_ef['emojie'].isin(['🏻','🏽','🏼','🏾','🏿'])][0:10]
    
    lista_resultado = [(row['emojie'], row['Cuenta']) for _, row in df_ef.iterrows()]

    return lista_resultado

**Resultado de ejecución del a función**

In [ ]:
q2_run_v2(file_path)

**Evaluación del tiempo de ejecución**

In [ ]:
cProfile.run('q2_run_v2(file_path)')

**Memoria total utilizada**

In [ ]:
np.sum(memory_usage((q2_run_v2, (file_path,))))

## Punto 3 influyentes

### **Análisis de mención a los usuarios**

Se evidencia que tanto en la columna content como en la columna mentionedUsers hat mención a los usuarios, por tanto, es necesario realizar la exploración aleatoria de si los usuarios mencionados en content con los mismo de mentionedUsers

In [ ]:
def qinicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)

    return df_inicial

resultado2 = qinicial(file_path)[['content','mentionedUsers']].copy().explode(column = 'mentionedUsers')
resultado2= resultado2[~resultado2['mentionedUsers'].isnull()].reset_index()
pd.set_option('display.max_colwidth', 500) 

display(resultado2)

Al observar el caso anterior se deja en evidencia casos como los de los registros de las fila 117388, el cual tiene indicados dos veces a la misma persona, en este caso se debería poder entender la necesidad del cliente final, dado que una peersona comentada dos veces en un mismo twit puede indicar que la persona es relevante para quien actual en "X", pero al tiempo podría mostrar un dato desviado, dado que si solo queremos conocer a quien se han etiquetado por comentario, pues no importa su número de veces etiquetado, se deberá contar una única vez.

Por el momento, aludiendo a que si hay una etiqueta múltiple para una misma cuenta en un solo comentario podría indicar un sentido de relevancia para quien redacta el mensaje, se trasladará esa urgencia al análisis de los más etiquetados, de tal forma que se obseve en la lista la urgencia o importancia que le dan los usuario de "X" a quienes etiquetan. 

### **Función inicial**

Dado la explicación anterior se va a hacer uso de la columna mentionedUsers para obtener la información de la mensión de usuarios dentro del contenido de una publicación.

In [16]:
def q3_run_inicial(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargarlo en un DataFrame
    data = []
    # Crear conjunto para la claves únicas 
    unique_keys = set()

    with open(file_path, 'r') as file:
        for line in file:
            json_data = json.loads(line.strip())
            data.append(json_data)
            unique_keys.update(json_data.keys())

    #Crear el dataframe inicial en función de los datos leídos y con columnas nombradas tal como las claves únicas obtenidas
    df_inicial = pd.DataFrame(data).reindex(columns=unique_keys)
    
    #Dado que se ha validado previamente que no hay twits duplicados, se puede usar la columna mentionedUsers, para ser explotada (explode) y luego expandida (json_normalize) y con esta hacer todo el procesamiento solicitado.
    
    df_relevantes =  json_normalize(pd.DataFrame(df_inicial['mentionedUsers']).dropna().explode(column='mentionedUsers').reset_index(drop=True)['mentionedUsers'])
    
    # Crear DataFrame con los emojis y su frecuencia
    df_relev = pd.DataFrame(Counter(df_relevantes['username']).most_common(), columns=['username', 'Cuenta'])
    
    lista_resultado = [(row['username'], row['Cuenta']) for _, row in df_relev.iterrows()][:10]
    
    return lista_resultado

**Resultado de ejecución del a función**

In [17]:
q3_run_inicial(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

**Evaluación del tiempo de ejecución**

In [18]:
cProfile.run('q3_run_inicial(file_path)')

         9856895 function calls (9722860 primitive calls) in 16.618 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.107    1.107   15.906   15.906 1861183408.py:1(q3_run_inicial)
        1    0.021    0.021    1.002    1.002 1861183408.py:23(<listcomp>)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
        3    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_2d)
        7    0.000    0.000    0.042    0.006 <__array_function__ internals>:177(concatenate)
       59    0.000    0.000    0.003    0.000 <__array_function__ internals>:177(copyto)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(delete)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:177(prod)
        3    0.000    0.000    0.042    

**Memoria total utilizada**

In [19]:
np.sum(memory_usage((q3_run_inicial, (file_path,))))

113474.8203125

## Código no optimizado

In [ ]:
fgdsghg

# Lista para almacenar los DataFrames individuales
data = []
lista_keys = []

# Abrir el archivo y leer línea por línea
with open(file_path, 'r') as file:
    for line in file:
        # Crear un diccionariodataframe con una sola columna "json" para cada línea
        diccionario_df = pd.DataFrame({'json': [line.strip()]})
        # Agregar el DataFrame a la lista
        data.append(diccionario_df)
        lista_keys.extend(list(diccionario_df['json'].apply(json.loads)[0].keys()))

In [ ]:
# Concatenar todos los encabezados únicos para luego formar un dataframe genetal
lista_keys_unicas = set(lista_keys)
#Crear data frame vacío para luego concatenar
df_inicial = pd.DataFrame(columns=lista_keys_unicas)
#Iterar para concatenar todos los df
for df in data: 
    df= df.reindex(columns=lista_keys_unicas)
    df_inicial= pd.concat([df_inicial,df])